# Bibliothèques

In [ ]:
import pickle
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Chargement des données

In [ ]:
with open('data/visages.pkl', 'rb') as fh:
    visages = pickle.load(fh)
    visages = visages.reshape(len(visages), -1)

with open('data/prenoms.pkl', 'rb') as fh:
    noms = pickle.load(fh)
    
with open('data/admis.pkl', 'rb') as fh:
    admis = pickle.load(fh)
    
print('Shape of visages matrix --> ', visages.shape)
print(set(noms)) 

# Création du modèle CNN

In [ ]:
# Initialisation du CNN
model = Sequential()

# Ajout des couches de convolution (extraction des caractéristiques à partir des images)
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Aplatir les données
model.add(Flatten())

# Ajout des couches denses (classification des images)
model.add(Dense(128, activation='relu'))
model.add(Dense(len(set(noms)), activation='softmax')) 

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Redimensionner les images des visages
visages = visages.reshape(len(noms), 50, 50, 3)

# Initialisation de l'encodeur de labels
label_encoder = LabelEncoder()

# Ajuster et transformer les labels pour produire des labels encodés
encoded_noms = label_encoder.fit_transform(noms)

# Redimensionner les labels encodés pour les utiliser avec OneHotEncoder
encoded_noms = encoded_noms.reshape(len(encoded_noms), 1)

# Initialisation de OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse_output=False)

# Ajuster et transformer les labels encodés pour produire des labels one-hot encodés
one_hot_noms = one_hot_encoder.fit_transform(encoded_noms)

# Utiliser les labels one-hot encodés lors de l'entraînement du modèle
model.fit(visages, one_hot_noms, epochs=10, batch_size=32, validation_split=0.2)

# Détection des visages en temps réel

In [ ]:
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
camera = cv2.VideoCapture(0)

while True:
    ret, frame = camera.read()
    
    if ret:
        coordonnees_visage = cascade_visage.detectMultiScale(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), 1.3, 5)

        for (x, y, l, h) in coordonnees_visage:
            visage = frame[y:y + h, x:x + l, :]
            visage_redimensionne = cv2.resize(visage, (50, 50)).reshape(1, 50, 50, 3)
            texte = label_encoder.inverse_transform([np.argmax(model.predict(visage_redimensionne))])
            if admis[noms.index(texte[0])]:
                cv2.putText(frame, texte[0] + " (Admis)", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                cv2.rectangle(frame, (x, y), (x + l, y + l), (0, 255, 0), 2)
            else:
                cv2.putText(frame, texte[0] + " (Non Admis)", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                cv2.rectangle(frame, (x, y), (x + l, y + l), (0, 0, 255), 2)
            
        cv2.imshow('Reconnaissance faciale en temps réel', frame)
        
        if cv2.waitKey(1) == 27:
            break
            
    else:
            print("erreur")
            break
            
cv2.destroyAllWindows()
camera.release()